# Import packages and get API key

In [3]:
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain_text_splitters import TokenTextSplitter
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnableParallel
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY,model_name="gpt-4o-mini")

# Read Moby-Dick.txt

In [1]:
with open("./Moby-Dick.txt", 'r', encoding='utf-8') as f:
    mobi_dick_book = f.read()

# Chain to brake text into chunks

In [5]:
text_chunks_chain = (
    RunnableLambda(lambda x: 
        [
            {
                'chunk': text_chunk, 
            }
            for text_chunk in 
               TokenTextSplitter(chunk_size=3000, chunk_overlap=100).split_text(x)
        ]
    )
)

In [6]:
summarize_chunk_prompt_template = """
Write a concise summary of the following text, and include the main details.
Text: {chunk}
"""
 
summarize_chunk_prompt = PromptTemplate.from_template(summarize_chunk_prompt_template)
summarize_chunk_chain = summarize_chunk_prompt | llm
 
summarize_map_chain = (
    RunnableParallel (
        {
            'summary': summarize_chunk_chain | StrOutputParser()        
        }
    )
) 

In [7]:
summarize_summaries_prompt_template = """
Write a coincise summary of the following text, which joins several summaries, and include the main details.
Text: {summaries}
"""
 
summarize_summaries_prompt = PromptTemplate.from_template(summarize_summaries_prompt_template)

In [8]:
summarize_reduce_chain = (
    RunnableLambda(lambda x: 
        {
            'summaries': '\n'.join([i['summary'] for i in x]), 
        })
    | summarize_summaries_prompt 
    | llm 
    | StrOutputParser()
)

In [9]:
map_reduce_chain = (
   text_chunks_chain
   | summarize_map_chain.map()
   | summarize_reduce_chain
) 

In [10]:
summary = map_reduce_chain.invoke(mobi_dick_book)

In [11]:
print(summary)

The Project Gutenberg eBook "Moby-Dick; or The Whale," by Herman Melville, was released in June 2001 and updated in August 2021. The narrative begins with the protagonist, Ishmael, expressing his motivations for going to sea, which include escaping gloom and the allure of adventure. As he travels to New Bedford, he is disappointed in not catching a boat to Nantucket and seeks lodging at the "Spouter Inn." Here, he encounters a chaotic oil painting that symbolizes the uncertainty of his impending voyage.

Ishmael shares a room with an unfamiliar harpooneer, Queequeg, provoking initial fear and curiosity due to Queequeg’s tattoos and unfamiliar customs. Despite his apprehensions, he slowly recognizes Queequeg as a fellow human being. Their interaction culminates in a tense but ultimately bonding experience as they begin to understand one another, despite cultural differences. The text explores themes of exploration, curiosity, and the complexities of human connections amid the backdrop o

# Summarizing across documents
### Wikipedia

In [12]:
from langchain.document_loaders import WikipediaLoader
 
wikipedia_loader = WikipediaLoader(query="Paestum", load_max_docs=2)
wikipedia_docs = wikipedia_loader.load()

In [13]:
print(wikipedia_docs)

[Document(metadata={'title': 'Paestum', 'summary': 'Paestum ( PEST-əm, US also  PEE-stəm, Latin: [ˈpae̯stũː]) was a major ancient Greek city on the coast of the Tyrrhenian Sea, in Magna Graecia. The ruins of Paestum are famous for their three ancient Greek temples in the Doric order dating from about 550 to 450 BCE that are in an excellent state of preservation. The city walls and amphitheatre are largely intact, and the bottom of the walls of many other structures remain, as well as paved roads. The site is open to the public, and there is a modern national museum within it, which also contains the finds from the associated Greek site of Foce del Sele.\nPaestum was established around 600 BCE by settlers from Sybaris, a Greek colony in southern Italy, under the name of Poseidonia (Ancient Greek: Ποσειδωνία). The city thrived as a Greek settlement for about two centuries, witnessing the development of democracy. In 400 BCE, the Lucanians seized the city. Romans took over in 273 BCE, ren

### .docx, .pdf, and .txt

In [14]:
from langchain.document_loaders import Docx2txtLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import TextLoader
 
word_loader = Docx2txtLoader("Paestum/Paestum-Britannica.docx")
word_docs = word_loader.load()
 
pdf_loader = PyPDFLoader("Paestum/PaestumRevisited.pdf")
pdf_docs = pdf_loader.load()
 
txt_loader = TextLoader("Paestum/Paestum-Encyclopedia.txt")
txt_docs = txt_loader.load()

### merge into list

In [15]:
all_docs = wikipedia_docs + word_docs + pdf_docs + txt_docs

In [16]:
doc_summary_template = """Write a concise summary of the following text:
{text}
DOC SUMMARY:"""
doc_summary_prompt = PromptTemplate.from_template(doc_summary_template)
 
doc_summary_chain = doc_summary_prompt | llm

In [17]:
refine_summary_template = """
Your must produce a final summary from the current refined summary
which has been generated so far and from the content of an additional document.
This is the current refined summary generated so far: {current_refined_summary}
This is the content of the additional document: {text}
Only use the content of the additional document if it is useful, 
otherwise return the current full summary as it is."""
 
refine_summary_prompt = PromptTemplate.from_template(refine_summary_template)
 
refine_chain = refine_summary_prompt | llm | StrOutputParser()

In [18]:
def refine_summary(docs):
 
    intermediate_steps = []
    current_refined_summary = ''
    for doc in docs:
        intermediate_step = \
           {"current_refined_summary": current_refined_summary, 
            "text": doc.page_content}
        intermediate_steps.append(intermediate_step)
        
        current_refined_summary = refine_chain.invoke(intermediate_step)
        
    return {"final_summary": current_refined_summary,
            "intermediate_steps": intermediate_steps}

In [19]:
full_summary = refine_summary(all_docs)

In [20]:
print(full_summary )

{'final_summary': "**Final Summary:**\n\nPaestum, an ancient Greek city located on the coast of the Tyrrhenian Sea in Magna Graecia, was established around 600 BCE by settlers from Sybaris, originally named Poseidonia. The city flourished for approximately two centuries as an autonomous Greek polis, developing a democratic system and enjoying intense cultural and commercial exchange with the Greek world and non-Greek populations, including the Etruscans. It was endowed with a defensive wall featuring four gates and saw the construction of three notable Doric temples in the sixth and fifth centuries BCE, often referred to as dedicated to Hera and Athena, although there has been some debate about the specific deities honored. A Greek agora was established north of the sanctuary of Hera, which included civic structures such as a bouleuterion.\n\nIn 400 BCE, control shifted to the Lucanians, an indigenous Italic people, who renamed the city Paestum. However, evidence indicates that during 

{'final_summary': "**Final Summary:**\n\nPaestum, an ancient Greek city located on the coast of the Tyrrhenian Sea in Magna Graecia, was established around 600 BCE by settlers from Sybaris, originally named Poseidonia. The city flourished for approximately two centuries as an autonomous Greek polis, developing a democratic system and enjoying intense cultural and commercial exchange with the Greek world and non-Greek populations, including the Etruscans. It was endowed with a defensive wall featuring four gates and saw the construction of three notable Doric temples in the sixth and fifth centuries BCE, often referred to as dedicated to Hera and Athena, although there has been some debate about the specific deities honored. A Greek agora was established north of the sanctuary of Hera, which included civic structures such as a bouleuterion.\n\nIn 400 BCE, control shifted to the Lucanians, an indigenous Italic people, who renamed the city Paestum. However, evidence indicates that during this period, the city continued to be referred to as Poseidonia by many scholars. Though traditional historical interpretations suggest that the transition from Poseidonia to Paestum involved a cultural transformation due to violent conquest, recent research suggests that this transition may have occurred gradually and peacefully. The archaeological record reveals that Poseidonia continued to prosper during the Lucanian period, with existing temples being maintained and worship practices persisting, including the construction of two stoai and an Asklepieion at the agora. Despite earlier beliefs that the Lucanians enslaved the Greeks, evidence shows that Greek practices, including language and civic structures, persisted during and after the shift in control. The name change may have been a later development, potentially related to the Roman conquest.\n\nThe city remained under Lucanian rule until 273 BCE, when it became a Roman colony. The process of Roman colonization followed a ritualistic tradition, involving specific founding ceremonies that established a connection to Rome's mythological origins. The city continued to thrive during the early Roman Empire but eventually succumbed to environmental changes, such as the silting up of the Silarus River leading to the creation of a malarial swamp, and was ultimately abandoned after being sacked by Muslim raiders in 871 CE.\n\nAfter the founding of the Roman colony, Paestum underwent significant changes, adapting its political and civic structures to align with Roman models while retaining elements of its Greek and Lucanian heritage. The Greek/Lucanian agora ceased operation as a new Roman forum was established, leading to the demolition of traditional buildings. However, sanctuaries continued to be respected and remained active sites of worship. During the Second Punic War, despite the dangers posed by Hannibal, Paestum demonstrated loyalty to Rome, which in turn granted the city a degree of autonomy, including the privilege of minting its own coins.\n\nToday, the well-preserved ruins of Paestum attract visitors to its archaeological site, which covers around 25 hectares of the original 120-hectare expanse. The three Doric temples constructed between 550 and 450 BCE are a significant draw, showcasing traditional Greek architectural features like massive colonnades with pronounced entasis. Notably, the Doric Order of Paestum is characterized by the most exaggerated entasis of any Antique example, with very wide squat capitals that emphasize a primitive effect. This distinctive architectural style was highly admired by Neo-Classical architects in the 18th and 19th centuries and was employed where powerful effects were desired. The site is surrounded by impressive defensive walls measuring roughly 4.75 km in length, with several gates leading in various directions.\n\nThe archaeological site is part of the modern frazione of Paestum in the comune of Capaccio Paestum, located in the Province of Salerno in Campania, Italy. Capaccio Paestum has developed into a popular seaside resort, featuring sandy beaches and convenient access to the Paestum railway station on the Naples-Salerno-Reggio Calabria railway line. The area also boasts a national museum that houses artifacts, including finds from nearby Greek sites like Foce del Sele. The rich history and geographical setting of Capaccio Paestum contribute significantly to the cultural significance of the area.", 'intermediate_steps': [{'current_refined_summary': '', 'text': 'Paestum ( PEST-əm, US also  PEE-stəm, Latin: [ˈpae̯stũː]) was a major ancient Greek city on the coast of the Tyrrhenian Sea, in Magna Graecia. The ruins of Paestum are famous for their three ancient Greek temples in the Doric order dating from about 550 to 450 BCE that are in an excellent state of preservation. The city walls and amphitheatre are largely intact, and the bottom of the walls of many other structures remain, as well as paved roads. The site is open to the public, and there is a modern national museum within it, which also contains the finds from the associated Greek site of Foce del Sele.\nPaestum was established around 600 BCE by settlers from Sybaris, a Greek colony in southern Italy, under the name of Poseidonia (Ancient Greek: Ποσειδωνία). The city thrived as a Greek settlement for about two centuries, witnessing the development of democracy. In 400 BCE, the Lucanians seized the city. Romans took over in 273 BCE, renaming it Paestum and establishing a Latin colony. Later, its decline ensued from shifts in trade routes and the onset of flooding and marsh formation. As Pesto or Paestum, the town became a bishopric (now only titular), but it was abandoned in the Early Middle Ages, and left undisturbed and largely forgotten until the eighteenth century.\nToday the remains of the city are found in the modern frazione of Paestum, which is part of the comune of Capaccio Paestum in the Province of Salerno in the region of Campania, Italy. The modern settlement, directly to the south of the archaeological site, is a popular seaside resort with long sandy beaches. The Paestum railway station on the Naples-Salerno-Reggio Calabria railway line is directly to the east of the ancient city walls. \n\n\n== Name ==\nThe Greek settlers who founded the city originally named it Poseidonia (Ancient Greek: Ποσειδωνία). It was eventually conquered by the local Lucanians and later the Romans. The Lucanians renamed it to Paistos and the Romans gave the city its current name. \n\n\n== Ancient ruins and features ==\n\nMuch of the most celebrated features of the site today are the three large temples in the Archaic version of the Greek Doric order, dating from about 550 to 450 BCE. All are typical of the period, with massive colonnades having a very pronounced entasis (widening as they go down), and very wide capitals resembling upturned mushrooms. Above the columns, only the second Temple of Hera retains most of its entablature, the other two having only the architrave in place.\nThese were dedicated to Hera and Athena (Juno and Minerva to the Romans), although previously they often have been identified otherwise, following eighteenth-century arguments. The two temples of Hera are right next to each other, while the Temple of Athena is on the other side of the town center. There were other temples, both Greek and Roman, which are far less well preserved. \nPaestum is far from any sources of good marble. Unsurprisingly, the three main temples had few stone reliefs, perhaps using painting instead. Painted terracotta was used for some detailed parts of the structure.  The large pieces of terracotta that have survived are in the museum.\n\nThe whole ancient city of Paestum covered an area of approximately 120 hectares.  Only the 25 hectares that contain the three main temples and the other main buildings have been excavated. The other 95 hectares remain on private land and have not been studied.\nThe ancient city was surrounded by defensive walls, which are largely intact. The walls are approximately 4.75 km (3 mi) long in its polygonal perimeter, typically 15 m (49 ft) high, and 5–7 m (16–23 ft) thick. Corresponding with the cardinal points, there were four main openings in the wall: Porta Sirena (east to the hills); Porta Giustizia (south, now to the modern village Paestum); Porta Marina (west to the sea); and Porta Aurea (north), which was later destroyed. Positioned along the wall were 24 square or round towers. There may have been as many as 28, but '}, {'current_refined_summary': '**Final Summary:**\n\nPaestum, an ancient Greek city located on the coast of the Tyrrhenian Sea in Magna Graecia, was established around 600 BCE by settlers from Sybaris, originally named Poseidonia. The city flourished for approximately two centuries, during which it developed a democratic system. In 400 BCE, it was seized by the Lucanians, who renamed it Paistos, and later became a Roman colony in 273 BCE, taking on the name Paestum. The city experienced a decline due to shifting trade routes and environmental changes, ultimately becoming abandoned in the Early Middle Ages.\n\nToday, the well-preserved ruins of Paestum, famous for three large Doric temples dating from 550 to 450 BCE, attract visitors to its archaeological site. The temples, dedicated to Hera and Athena, showcase traditional Greek architectural features, including massive colonnades with pronounced entasis. Only a small portion of the ancient city has been excavated, with the main temples and several other structures covering approximately 25 hectares of the original 120-hectare site, while the remaining land remains largely unexplored.\n\nPaestum is enclosed by impressive defensive walls that are still largely intact, measuring about 4.75 km in length, with several main gates leading in various directions. Today, the archaeological site is part of the modern frazione of Paestum in the comune of Capaccio Paestum, within the Province of Salerno in Campania, Italy. The modern settlement has developed into a popular seaside resort, featuring sandy beaches and access to the Paestum railway station on the Naples-Salerno-Reggio Calabria railway line. A national museum within the site houses artifacts, including finds from the nearby Greek site of Foce del Sele.', 'text': 'Capaccio Paestum (formerly only Capaccio, from Latin Caput Aquae) is a town and comune in the province of Salerno in the Campania region of south-western Italy.  The ruins of the ancient Greek city of Paestum lie within borders of the comune.\n\n\n== History ==\n\n\n== Geography ==\nLocated in northern Cilento, near the mouth of Sele, Capaccio is a hill town surrounded by a plain in which resides almost all of the hamlets (frazioni) and the majority of the population, mostly concentrated at Capaccio Scalo, seat of the train station.\nThe municipality borders with Agropoli, Albanella, Cicerale, Eboli, Giungano, Roccadaspide and Trentinara. The hamlets are Borgo Nuovo, Capaccio Scalo, Cafasso, Chiorbo, Foce Sele, Gaiarda, Gromola, Laura, Licinella, Linora, Paestum, Ponte Barizzo, Rettifilo-Vannulo, Spinazzo, Santa Venere, Tempa di Lepre, Torre di Mare, Tempa San Paolo, and Vuccolo Maiorano.\n\n\n== Transport ==\nThe nearest airport is Salerno-Pontecagnano (QSR), 35 km from Capaccio.\n\n\n== Notable people ==\nVincenzo Romano\nMichele Siano\n\n\n== See also ==\nCilentan Coast\nCilento\n\n\n== Notes and references ==\n\n\n== External links =='}, {'current_refined_summary': "**Final Summary:**\n\nPaestum, an ancient Greek city located on the coast of the Tyrrhenian Sea in Magna Graecia, was established around 600 BCE by settlers from Sybaris, originally named Poseidonia. The city flourished for approximately two centuries, during which it developed a democratic system. In 400 BCE, it was seized by the Lucanians, who renamed it Paistos, and later became a Roman colony in 273 BCE, taking on the name Paestum. The city experienced a decline due to shifting trade routes and environmental changes, ultimately becoming abandoned in the Early Middle Ages.\n\nToday, the well-preserved ruins of Paestum, famous for three large Doric temples dating from 550 to 450 BCE, attract visitors to its archaeological site. The temples, dedicated to Hera and Athena, showcase traditional Greek architectural features, including massive colonnades with pronounced entasis. Only a small portion of the ancient city has been excavated, with the main temples and several other structures covering approximately 25 hectares of the original 120-hectare site, while the remaining land remains largely unexplored.\n\nPaestum is enclosed by impressive defensive walls that are still largely intact, measuring about 4.75 km in length, with several main gates leading in various directions. The archaeological site is part of the modern frazione of Paestum in the comune of Capaccio Paestum, located in the Province of Salerno in Campania, Italy. Capaccio Paestum is a hill town surrounded by a plain, with the majority of its population concentrated in Capaccio Scalo, the seat of the train station, and well connected by transport to nearby towns and the nearest airport at Salerno-Pontecagnano.\n\nThe modern settlement has developed into a popular seaside resort, featuring sandy beaches and access to the Paestum railway station on the Naples-Salerno-Reggio Calabria railway line. A national museum within the site houses artifacts, including finds from the nearby Greek site of Foce del Sele. The rich history and geographical setting of Capaccio Paestum contribute significantly to the area's cultural significance.", 'text': 'Paestum\n\nancient city, Italy\n\nPrint\xa0Cite\xa0Share\xa0Feedback\xa0\n\nAlso known as: Poseidonia\n\nWritten and fact-checked by\xa0\n\n\n\n\n\nThe Editors of Encyclopaedia Britannica\n\nLast Updated:\xa0Article History\n\n\n\n\n\nPaestum: Temple of Apollo\n\nSee all media\n\nCategory:\xa0Geography & Travel\n\nGreek:\xa0Poseidonia\n\nRelated Places:\n\n\xa0\n\nItaly\xa0ancient Greece\xa0Basilicata\n\nSee all related content →\n\nPaestum, Greek\xa0Poseidonia, ancient city in southern\xa0Italy\xa0near the west coast, 22 miles (35 km) southeast of modern\xa0Salerno\xa0and 5 miles (8 km) south of the Sele (ancient Silarus) River. Paestum is noted for its splendidly preserved Greek temples.\n\n\n\n\n\nVisit the ruins of the ancient Greek colony of Paestum and discover its history, culture, and society\n\nSee all videos for this article\n\nPoseidonia was probably founded about 600\xa0BC\xa0by Greek colonists from\xa0Sybaris, along the\xa0Gulf of Taranto, and it had become a flourishing town by 540, judging from its temples. After many years’ resistance the city came under the domination of the\xa0Lucanians\xa0(an\xa0indigenous\xa0Italic people) sometime before 400\xa0BC, after which its name was changed to Paestum. Alexander, the king of Epirus, defeated the Lucanians at Paestum about 332\xa0BC, but the city remained Lucanian until 273, when it came under Roman rule and a Latin colony was founded there. The city supported\xa0Rome\xa0during the\xa0Second Punic War. The locality was still prosperous during the early years of the Roman Empire, but the gradual silting up of the mouth of the Silarus River eventually created a malarial swamp, and Paestum was finally deserted after being sacked by Muslim raiders in\xa0AD\xa0871. The abandoned site’s remains were rediscovered in the 18th century.\n\n\n\nBritannica Quiz\n\nAncient Greece\n\n\n\n\n\nPaestum, Italy: Temple of Athena\n\nThe ancient Greek part of Paestum consists of two sacred areas containing three Doric temples in a remarkable state of preservation. During the ensuing Roman period a typical\xa0forum\xa0and town\xa0layout\xa0grew up between the two ancient Greek sanctuaries. Of the three temples, the\xa0Temple of Athena\xa0(the so-called Temple of Ceres) and the Temple of\xa0Hera\xa0I (the so-called Basilica) date from the 6th century\xa0BC, while the Temple of Hera II (the so-called Temple of Neptune) was probably built about 460\xa0BC\xa0and is the best preserved of the three. The Temple of Peace in the forum is a Corinthian-Doric building begun perhaps in the 2nd century\xa0BC. Traces of a Roman amphitheatre and other buildings, as well as intersecting main streets, have also been found. The circuit of the town walls, which are built of travertine blocks and are 15–20 feet (5–6 m) thick, is about 3 miles (5 km) in circumference. In July 1969 a farmer uncovered an ancient Lucanian tomb that contained Greek frescoes painted in the early\xa0classical style. Paestum’s archaeological museum contains these and other\xa0treasures\xa0from the site.'}, {'current_refined_summary': '**Final Summary:**\n\nPaestum, an ancient Greek city located on the coast of the Tyrrhenian Sea in Magna Graecia, was established around 600 BCE by settlers from Sybaris, originally named Poseidonia. The city flourished for approximately two centuries, developing a democratic system and becoming a significant coastal town by 540 BCE. In 400 BCE, it came under the control of the Lucanians, an indigenous Italic people, who renamed it Paestum. Despite a brief liberation by Alexander, king of Epirus, around 332 BCE, the city remained under Lucanian rule until 273 BCE, when it became a Roman colony, continuing to thrive during the early Roman Empire. However, it eventually fell victim to environmental changes, such as the silting up of the Silarus River creating a malarial swamp, and was finally abandoned after being sacked by Muslim raiders in 871 CE.\n\nToday, the well-preserved ruins of Paestum, renowned for three large Doric temples constructed between 550 and 450 BCE, attract visitors to its archaeological site. The temples, dedicated to Hera and Athena, display traditional Greek architectural features, including massive colonnades with pronounced entasis. Currently, only a fraction of the ancient city has been excavated, with the main temples and several other structures covering around 25 hectares of the original 120-hectare site; the remainder remains largely unexplored.\n\nPaestum is surrounded by impressive defensive walls, measuring roughly 4.75 km in length, with several gates leading in different directions. The archaeological site is part of the modern frazione of Paestum in the comune of Capaccio Paestum, located in the Province of Salerno in Campania, Italy. Capaccio Paestum has developed into a popular seaside resort, featuring sandy beaches and convenient access to the Paestum railway station on the Naples-Salerno-Reggio Calabria railway line. The area also boasts a national museum housing artifacts, including finds from the nearby Greek site of Foce del Sele. The rich history and geographical setting of Capaccio Paestum significantly contribute to the cultural significance of the area.', 'text': '16 \nwho came across the place before the historical Greek settlement was founded.53 Strabo, however, does claim, similarly to Aristotle, that it was the Achaeans who founded Sybaris and he also describes the mother colony, Sybaris, and mentions the name of its founder: Is of Helice.54       The Greeks flourished at Poseidonia for some 200 years. The city enjoyed the status of an autonomous Greek polis and was endowed with a defensive wall with four gates, probably built in phases. Inside the city walls three Doric-style temples were erected in the sixth and fifth centuries. There has been some disagreement on which gods the temples were dedicated too, however, even though within the scope of this study they are referred to as Hera I, Hera II and the Temple of Athena, they might as well have been dedicated to more than one god. A Greek agora was also laid out north of the sanctuary of Hera (Fig. 6). It was adorned with a variety of Greek features; among them a bouleuterion or possible ekklesiasterion.55 An intriguing structure identified as a heroon, a place that has been interpreted as being dedicated to founder of the city, was built on the western edge of the agora around 510-500 BC.56 Outside the walls of the city a possible harbour, its existence and position is however debated, several nekropoleis and extramural cult sites were located; of which one will be the focus of this study: The Sanctuary of Santa Venera south of the city wall. Poseidonia came to enjoy intense cultural and commercial exchange with the rest of the Greek world as well as non-Greek populations like the Etruscans to the north and other Italic tribes from the hinterland. The Etruscan influence combined with the Greek and local Poseidonian style can for example be seen in the very famous fifth century tomb known as Tomba del Tuffatore, the Tomb of the Diver.57 Altogether Poseidonia prospered  and continued doing so. When its mother colony, Sybaris, fell in 510 BC Poseidonia along with other colonies filled the gap as a commercial and cultural centre.58 However, as the fifth century closed, changes loomed on the horizon and the Greek settlement at Poseidonia was to be forever marked by what was to come.   2.1.2. The Lucanian period There were many subdivisions of the Italic population and among them were the Lucanians; an Oscan-speaking people with an already established close contact with the Greek cities of Magna                                                 53 Pedley 1990, 30.  54 Strab1. 6.6.13. 55 Pedley 1990, 79. 56 Pedley 1990, 38-39. 57 Mello 1985, 14. 58 Mello 1985, 14.'}, {'current_refined_summary': '**Final Summary:**\n\nPaestum, an ancient Greek city located on the coast of the Tyrrhenian Sea in Magna Graecia, was established around 600 BCE by settlers from Sybaris, originally named Poseidonia. The city flourished for approximately two centuries as an autonomous Greek polis, developing a democratic system and enjoying intense cultural and commercial exchange with the Greek world and non-Greek populations, including the Etruscans. It was endowed with a defensive wall, featuring four gates, and saw the construction of three notable Doric temples in the sixth and fifth centuries BCE, often referred to as dedicated to Hera and Athena, although there has been some debate about the specific deities honored. A Greek agora was established north of the sanctuary of Hera, which included civic structures such as a bouleuterion.\n\nIn 400 BCE, control shifted to the Lucanians, an indigenous Italic people, who renamed the city Paestum. The city remained under Lucanian rule until 273 BCE, when it became a Roman colony, continuing to thrive during the early Roman Empire. Nonetheless, it eventually succumbed to environmental changes, such as the silting up of the Silarus River leading to the creation of a malarial swamp, and was ultimately abandoned after being sacked by Muslim raiders in 871 CE.\n\nToday, the well-preserved ruins of Paestum attract visitors to its archaeological site, which covers around 25 hectares of the original 120-hectare expanse. The three Doric temples constructed between 550 and 450 BCE are a significant draw, showcasing traditional Greek architectural features like massive colonnades with pronounced entasis. The site is surrounded by impressive defensive walls measuring roughly 4.75 km in length, with several gates leading in various directions. \n\nThe archaeological site is part of the modern frazione of Paestum in the comune of Capaccio Paestum, located in the Province of Salerno in Campania, Italy. Capaccio Paestum has developed into a popular seaside resort, featuring sandy beaches and convenient access to the Paestum railway station on the Naples-Salerno-Reggio Calabria railway line. The area also boasts a national museum that houses artifacts, including finds from nearby Greek sites like Foce del Sele. The rich history and geographical setting of Capaccio Paestum contribute significantly to the cultural significance of the area.', 'text': '17 \nGraecia. 59 The Lucanians, the local name for the south Italian Samnite people, occupied the inland area between the Ionian and the Tyrrhenian coast in the fourth century BC. Samnite people were expanding in numbers and in need of new homes, and therefore looked to the prosperous Greek and Etruscan coastal cities. At the end of the fifth century cities like Cumae and Capua had been captured by the Samnites and around 400 BC also Poseidonia came to be under control of the Lucanians.60 This is the traditional interpretation among scholars. However, in the final discussion of this study I hope to nuance this interpretation a bit.  Consequently, there has been a strong belief among many scholars that a “profound cultural change took place in the second half of the fifth century” at Poseidonia as a result of this Lucanian invasion.61 However, recent research has showed us otherwise. It is clear that Poseidonia at some point became a Lucanian city and that this happened gradually and peacefully. Much of the earlier historical interpretation of the Lucanian overtaking of Poseidonia derives from a statement made by Aristoxenos of Tarentum, a pupil of Aristoteles writing in the fourth century BC. He claims that the Greeks were essentially enslaved by the “Tyrrhenians”. This have led scholars to blame the Lucanians for this supposed regression in culture, politics and economy. The “Tyrrhenians” in this case have been interpreted as the Lucanians but it might as well refer to the Romans, Campanians, another Samnite people or the Etruscans; the latter have in fact in ancient literature been generally referred to as “Tyrrhenians”.62 The pessimistic literary evidence concerning the new trends in Poseidonia does, however, not have much support in the archaeological material. The supposed subjugation of the Greeks and the violent conquest of the city may in fact rather be a reflection of Greek attitudes in Tarentum or maybe even part of a larger process going on in Southern Italy at the time: Lucanization.63 But the evidence from Poseidonia suggests that the Greek practises continued; the bouleuterion and the heroon remained in use and so did the Greek sanctuaries and the Greek language.64 Some scholars claim that the change of the Greek name Poseidonia to Paistos or Paistom (later Paestum) can be seen as evidence of a conflict between Greeks and Lucanians, if this renaming happened under the Lucanian rule. The exact time of the change is however unsure, it might as well be a result of the later Roman conquest.65 Pedley does, on the contrary, not agree with the renaming \n                                                59 Fracchia & Gualtieri 1989, 217. 60 Pedley 1990, 97. 61 Pedley 1990, 97; Horsnaes 2002, 11. 62 Isayev 2007, 17-18. 63 Isayev 2007, 18-19; Zuchtriegel 2016, 14.  64 Pedley 1990, 97. 65 Mello 1985, 15.'}, {'current_refined_summary': '**Final Summary:**\n\nPaestum, an ancient Greek city located on the coast of the Tyrrhenian Sea in Magna Graecia, was established around 600 BCE by settlers from Sybaris, originally named Poseidonia. The city flourished for approximately two centuries as an autonomous Greek polis, developing a democratic system and enjoying intense cultural and commercial exchange with the Greek world and non-Greek populations, including the Etruscans. It was endowed with a defensive wall featuring four gates and saw the construction of three notable Doric temples in the sixth and fifth centuries BCE, often referred to as dedicated to Hera and Athena, although there has been some debate about the specific deities honored. A Greek agora was established north of the sanctuary of Hera, which included civic structures such as a bouleuterion.\n\nIn 400 BCE, control shifted to the Lucanians, an indigenous Italic people, who renamed the city Paestum. Traditional historical interpretations suggest that the shift from Poseidonia to Paestum involved a cultural transformation due to violent conquest, but recent research indicates that this transition may have occurred gradually and peacefully. Despite earlier beliefs that the Lucanians enslaved the Greeks and led to a cultural decline, archaeological evidence shows that Greek practices, including language and civic structures, persisted during and after the shift in control. The name change may have been a later development, potentially related to the Roman conquest.\n\nThe city remained under Lucanian rule until 273 BCE, when it became a Roman colony, continuing to thrive during the early Roman Empire. Nonetheless, it eventually succumbed to environmental changes, such as the silting up of the Silarus River leading to the creation of a malarial swamp, and was ultimately abandoned after being sacked by Muslim raiders in 871 CE.\n\nToday, the well-preserved ruins of Paestum attract visitors to its archaeological site, which covers around 25 hectares of the original 120-hectare expanse. The three Doric temples constructed between 550 and 450 BCE are a significant draw, showcasing traditional Greek architectural features like massive colonnades with pronounced entasis. The site is surrounded by impressive defensive walls measuring roughly 4.75 km in length, with several gates leading in various directions.\n\nThe archaeological site is part of the modern frazione of Paestum in the comune of Capaccio Paestum, located in the Province of Salerno in Campania, Italy. Capaccio Paestum has developed into a popular seaside resort, featuring sandy beaches and convenient access to the Paestum railway station on the Naples-Salerno-Reggio Calabria railway line. The area also boasts a national museum that houses artifacts, including finds from nearby Greek sites like Foce del Sele. The rich history and geographical setting of Capaccio Paestum contribute significantly to the cultural significance of the area.', 'text': '18 \nand states that the city continued to be called Poseidonia during the Lucanian period.66 In the present study I too will continue to use the name Poseidonia, in line with most scholars.       During the fourth century BC and up to the establishing of the Latin colony in 273 BC Poseidonia prospered as revealed in the archaeological record. Although there were no major large-scale building programmes to match the constructions of the Greek temples and sanctuaries those places continued to be looked after and the gods were worshiped by the new inhabitants.67 However, two stoai were built at the Greek agora as well as an Asklepieion (Table). The changes that came along with the Lucanians are to be seen in the agriculturalization and the density of settlement of the territory of Poseidonia as well as new burial practices in terms of extraordinary decorations.68   2.1.3. The Latin colony In 273 BC the expanding city of Rome had reached past the river Sele and stood at the gates of the Greek/Lucanian city of Poseidonia. Rome had been expanding for years, and the Republic was founding colonies, ex novo or, in the case of Poseidonia-Paestum, taking over already established settlements, all over the Italian peninsula. The Roman colonies, perhaps especially the ones that were founded during the late Republic, were behaving in a strikingly uniform way. The Roman, or the Latin, colony, is well known for its formulaic way of structure and features; in other words, we know the nature of a Roman colony without having to see one. The foundation ritual has been described in ancient literary sources and is said to originate from the time of Rome’s mythological foundation when Romulus, 753 BC, with permission of the gods laid out the sacred boundary, pomerium, around the city.69 The founding ritual of the colonies is describes by Bispham in the article ‘Coloniam deducere: how Roman was the Roman colonisation during the middle Republic?’:   Firstly, the deductio, or marching out, of the colonists from Rome to the site of the colony; the deductor (founder), in ritual dress (the cinctus Gabinus), ploughs the sulcus primigenius (the original furrow), tracing the line of the future walls and instantiating a ritual barrier, the pomoerium.70    In other words, it is clear that there was a model for Roman colonization and that the newly founded colonies were using the same set of rules and rituals to express their relationship, as                                                 66 Pedley 1990, 97.  67 Pedley 1990, 112. 68 Pedley 1990, 97. 69 Plut. Vit. Rom. 11.1-4. 70 Bispham 2006, 74.'}, {'current_refined_summary': "**Final Summary:**\n\nPaestum, an ancient Greek city located on the coast of the Tyrrhenian Sea in Magna Graecia, was established around 600 BCE by settlers from Sybaris, originally named Poseidonia. The city flourished for approximately two centuries as an autonomous Greek polis, developing a democratic system and enjoying intense cultural and commercial exchange with the Greek world and non-Greek populations, including the Etruscans. It was endowed with a defensive wall featuring four gates and saw the construction of three notable Doric temples in the sixth and fifth centuries BCE, often referred to as dedicated to Hera and Athena, although there has been some debate about the specific deities honored. A Greek agora was established north of the sanctuary of Hera, which included civic structures such as a bouleuterion.\n\nIn 400 BCE, control shifted to the Lucanians, an indigenous Italic people, who renamed the city Paestum. However, evidence indicates that during this period, the city continued to be referred to as Poseidonia by many scholars. Though traditional historical interpretations suggest that the transition from Poseidonia to Paestum involved a cultural transformation due to violent conquest, recent research indicates that this transition may have occurred gradually and peacefully. The archaeological record reveals that Poseidonia continued to prosper during the Lucanian period, with existing temples being maintained and worship practices persisting, including the construction of two stoai and an Asklepieion at the agora. Despite earlier beliefs that the Lucanians enslaved the Greeks, evidence shows that Greek practices, including language and civic structures, persisted during and after the shift in control. The name change may have been a later development, potentially related to the Roman conquest.\n\nThe city remained under Lucanian rule until 273 BCE, when it became a Roman colony. The process of Roman colonization followed a ritualistic tradition, involving specific founding ceremonies that established a connection to Rome's mythological origins. The city continued to thrive during the early Roman Empire but eventually succumbed to environmental changes, such as the silting up of the Silarus River leading to the creation of a malarial swamp, and was ultimately abandoned after being sacked by Muslim raiders in 871 CE.\n\nToday, the well-preserved ruins of Paestum attract visitors to its archaeological site, which covers around 25 hectares of the original 120-hectare expanse. The three Doric temples constructed between 550 and 450 BCE are a significant draw, showcasing traditional Greek architectural features like massive colonnades with pronounced entasis. The site is surrounded by impressive defensive walls measuring roughly 4.75 km in length, with several gates leading in various directions.\n\nThe archaeological site is part of the modern frazione of Paestum in the comune of Capaccio Paestum, located in the Province of Salerno in Campania, Italy. Capaccio Paestum has developed into a popular seaside resort, featuring sandy beaches and convenient access to the Paestum railway station on the Naples-Salerno-Reggio Calabria railway line. The area also boasts a national museum that houses artifacts, including finds from nearby Greek sites like Foce del Sele. The rich history and geographical setting of Capaccio Paestum contribute significantly to the cultural significance of the area.", 'text': '19 \nwell as their obedience and loyalty to their mother colony: Rome.71 However, Poseidonia, renamed Paestum after the founding of the Latin colony inside its walls, being an early example of Roman colonization as well as an already established city with all its functions and features, this ‘model of colonization’ cannot be applied in the same way. As the following analysis of Poseidonia-Paestum will show, Rome was the model for the colony’s political system. The rôle of the city was, however, adapted to fit its purposes as a colony as well as a product of new and previous influence and it was thus not a direct replica.72       After the founding of the Latin colony the city was altered radically (Table). The Greek/Lucanian agora ceased to be in use and instead a Roman forum was laid out in the southern part of the old agora. Traditional Roman buildings, especially those relating to political and juridical aspects, were constructed at the new forum and simultaneously the buildings with similar functions at the agora were demolished. The sanctuaries, however, remained in use, both the intramural and the extramural. They were respected by the new settlers and continued to be objects of worship.73 A striking continuity of divine worshipping is analysed in detail in the further analysis of the Sanctuary of Santa Venera.      The bonds with the mother colony, the city of Rome, were strong; both in a juridical and political sense as well as affectional. Paestum supported Rome in times of need and aligned itself with its mother colony politically and diplomatically. When the Carthaginian general Hannibal was roaming around southern Italy during the second Punic War, 218-201 BC, and literally breathing down the necks of the Paestans, the city still remained loyal to Rome. However, the many wars and the feeling of unsafety these times must have given rise to might have been a hard blow on the Paestan society. That said, it is a well-known fact that nothing binds a society together like a common enemy, and Paestum was thus rewarded for its loyalty. A certain degree of autonomy was given to the city and also the privilege of striking its own coins.74 Thus, Rome continued to be a strong influencer on the Paestan society although it is evident, which is shown in the further analysis of some aspects of the nature of Roman influence, that the city never forgot its Greek and Lucanian roots. In 133 the via Popilia, running to the east of Paestum, was constructed and as a result of this the city was unfortunately cut off from principal routes connecting the city to the rest of the peninsula.75                                                            71 Bispham 2006, 73-74. 72 Sewell 2010, 86. 73 Pedley 1990, 113. 74 Pedley 1990, 113. 75 Pedley 1990, 113-114.'}, {'current_refined_summary': "**Final Summary:**\n\nPaestum, an ancient Greek city located on the coast of the Tyrrhenian Sea in Magna Graecia, was established around 600 BCE by settlers from Sybaris, originally named Poseidonia. The city flourished for approximately two centuries as an autonomous Greek polis, developing a democratic system and enjoying intense cultural and commercial exchange with the Greek world and non-Greek populations, including the Etruscans. It was endowed with a defensive wall featuring four gates and saw the construction of three notable Doric temples in the sixth and fifth centuries BCE, often referred to as dedicated to Hera and Athena, although there has been some debate about the specific deities honored. A Greek agora was established north of the sanctuary of Hera, which included civic structures such as a bouleuterion.\n\nIn 400 BCE, control shifted to the Lucanians, an indigenous Italic people, who renamed the city Paestum. However, evidence indicates that during this period, the city continued to be referred to as Poseidonia by many scholars. Though traditional historical interpretations suggest that the transition from Poseidonia to Paestum involved a cultural transformation due to violent conquest, recent research indicates that this transition may have occurred gradually and peacefully. The archaeological record reveals that Poseidonia continued to prosper during the Lucanian period, with existing temples being maintained and worship practices persisting, including the construction of two stoai and an Asklepieion at the agora. Despite earlier beliefs that the Lucanians enslaved the Greeks, evidence shows that Greek practices, including language and civic structures, persisted during and after the shift in control. The name change may have been a later development, potentially related to the Roman conquest.\n\nThe city remained under Lucanian rule until 273 BCE, when it became a Roman colony. The process of Roman colonization followed a ritualistic tradition, involving specific founding ceremonies that established a connection to Rome's mythological origins. The city continued to thrive during the early Roman Empire but eventually succumbed to environmental changes, such as the silting up of the Silarus River leading to the creation of a malarial swamp, and was ultimately abandoned after being sacked by Muslim raiders in 871 CE.\n\nAfter the founding of the Roman colony, Paestum underwent significant changes, adapting its political and civic structures to align with Roman models while retaining elements of its Greek and Lucanian heritage. The Greek/Lucanian agora ceased operation as a new Roman forum was established, leading to the demolition of traditional buildings. However, sanctuaries continued to be respected and remained active sites of worship. During the Second Punic War, despite the dangers posed by Hannibal, Paestum demonstrated loyalty to Rome, which in turn granted the city a degree of autonomy, including the privilege of minting its own coins.\n\nToday, the well-preserved ruins of Paestum attract visitors to its archaeological site, which covers around 25 hectares of the original 120-hectare expanse. The three Doric temples constructed between 550 and 450 BCE are a significant draw, showcasing traditional Greek architectural features like massive colonnades with pronounced entasis. The site is surrounded by impressive defensive walls measuring roughly 4.75 km in length, with several gates leading in various directions.\n\nThe archaeological site is part of the modern frazione of Paestum in the comune of Capaccio Paestum, located in the Province of Salerno in Campania, Italy. Capaccio Paestum has developed into a popular seaside resort, featuring sandy beaches and convenient access to the Paestum railway station on the Naples-Salerno-Reggio Calabria railway line. The area also boasts a national museum that houses artifacts, including finds from nearby Greek sites like Foce del Sele. The rich history and geographical setting of Capaccio Paestum contribute significantly to the cultural significance of the area.", 'text': 'Paestum. Greek colony in Italy, south of Naples, of which a group of ruined Doric temples survives (c.530bc–c.460 bc). This Doric Order has the most exaggerated entasis of any Antique example, and the very wide squat capitals on top of the shafts emphasize the primitive effect. The Paestum Order was much admired by Neo-Classical architects in C18 and C19, and was used where powerful effects were sought.\n\nBibliography\n\nJ. Curl (2001);\nDinsmoor (1950);\nC. Normand (1852);'}]}